In [1]:
import IPython.display as ipd

import numpy as np
import torch


import stft_64 as stft
from audio_processing import griffin_lim

from scipy.io.wavfile import read

#### Setup hparams

In [2]:
class hparams_class:
    def __init__(self):
        self.temp=False
hparams = hparams_class()
hparams.sampling_rate = 22050

In [3]:
def load_wav_to_torch(full_path):
    sampling_rate, data = read(full_path)
    return torch.DoubleTensor(data.astype(np.float32)), sampling_rate

audio_origin, sampling_rate = load_wav_to_torch('demo.wav')
print(sampling_rate)
audio_origin = audio_origin.unsqueeze(0)
ipd.Audio(audio_origin[0].data.cpu().numpy(), rate=hparams.sampling_rate)

22050


In [4]:
stft_fn = stft.STFT(filter_length=1024, hop_length=256, win_length=1024,
                    window='hann')
stft_fn = stft.STFT(filter_length=1024, hop_length=256, win_length=1024,
                    window=None)


def compare(a,b):
    return torch.mean(torch.abs(a-b)), torch.mean((a-b)*(a-b))

def compare_L1(ori,gen):
    return torch.mean(torch.abs(ori-gen)/torch.abs(ori))


def compare_L2(a,b):
    return torch.sum(torch.abs(a-b)), torch.sum((a-b)*(a-b))

/mlx_devbox/users/yuliang.ji/playground/Exact_reconstruction/stft_64.py:67: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)


In [5]:
magnitude, phase_origin = stft_fn.transform(audio_origin)
print(magnitude.shape)

reconstruction = stft_fn.inverse(magnitude, phase_origin)
print(compare(reconstruction, audio_origin))
ipd.Audio(reconstruction[0].data.cpu().numpy(), rate=hparams.sampling_rate)

torch.Size([1, 513, 386])
(tensor(5.5155e-06, dtype=torch.float64), tensor(2.1955e-08, dtype=torch.float64))


In [8]:
def griffin_lim(magnitudes, stft_fn, n_iters=30):
    
    angles = np.angle(np.exp(2j * np.pi * np.random.rand(*magnitudes.size())))
    angles = angles.astype(np.float32)
    angles = torch.autograd.Variable(torch.from_numpy(angles))
    signal = stft_fn.inverse(magnitudes, angles).squeeze(1)
    # f=open('griffin lim.txt','a')
    for i in range(n_iters):
        if (i+1)%10==0:
            # f.write('%d/%d:%.4f'%(i,n_iters,compare_L1(magnitude,MAG)))
            a1,a2 = compare(signal, audio_origin)
            print('%d/%d:%.4f, %.4f'%(i,n_iters,a1,a2))
        MAG, angles = stft_fn.transform(signal)
        signal = stft_fn.inverse(magnitudes, angles).squeeze(1)
    return angles

phase_griffin_lim_30 =  griffin_lim(magnitude, stft_fn, n_iters=30)
phase_griffin_lim =  griffin_lim(magnitude, stft_fn, n_iters=500)
print(phase_griffin_lim.shape)
audio_griffin_lim = stft_fn.inverse(magnitude, phase_griffin_lim)
audio_griffin_lim = audio_griffin_lim.squeeze(0)
print(audio_griffin_lim.shape)

ipd.Audio(audio_griffin_lim[0].data.cpu().numpy(), rate=hparams.sampling_rate)

9/30:0.0658, 0.0118
19/30:0.0661, 0.0121
29/30:0.0662, 0.0122
9/500:0.0630, 0.0109
19/500:0.0629, 0.0111
29/500:0.0627, 0.0112
39/500:0.0625, 0.0112
49/500:0.0624, 0.0112
59/500:0.0623, 0.0112
69/500:0.0622, 0.0112
79/500:0.0621, 0.0112
89/500:0.0621, 0.0112
99/500:0.0621, 0.0112
109/500:0.0621, 0.0112
119/500:0.0620, 0.0112
129/500:0.0620, 0.0112
139/500:0.0619, 0.0112
149/500:0.0619, 0.0112
159/500:0.0619, 0.0112
169/500:0.0618, 0.0112
179/500:0.0618, 0.0112
189/500:0.0618, 0.0112
199/500:0.0617, 0.0112
209/500:0.0617, 0.0112
219/500:0.0616, 0.0112
229/500:0.0616, 0.0112
239/500:0.0615, 0.0111
249/500:0.0615, 0.0111
259/500:0.0614, 0.0111
269/500:0.0614, 0.0111
279/500:0.0613, 0.0111
289/500:0.0613, 0.0111
299/500:0.0612, 0.0111
309/500:0.0612, 0.0111
319/500:0.0611, 0.0111
329/500:0.0611, 0.0111
339/500:0.0610, 0.0111
349/500:0.0610, 0.0110
359/500:0.0609, 0.0110
369/500:0.0609, 0.0110
379/500:0.0608, 0.0110
389/500:0.0608, 0.0110
399/500:0.0608, 0.0110
409/500:0.0607, 0.0110
419/50

In [7]:
magnitude_gl, phase_gl = stft_fn.transform(audio_griffin_lim)
print(magnitude_gl.shape)
print(compare_L1(magnitude,magnitude_gl))

torch.Size([1, 513, 386])
tensor(0.1307, dtype=torch.float64)


In [8]:
def newton_iterative(magnitudes, stft_fn, n_iters=30, angles=None):
    
    if angles==None:
        angles = np.angle(np.exp(2j * np.pi * np.random.rand(*magnitudes.size())))
        angles = angles.astype(np.float32)
        angles = torch.autograd.Variable(torch.from_numpy(angles))

    signal = stft_fn.inverse(magnitudes, angles).squeeze(1)
    MAG, angles_1 = stft_fn.transform(signal)
        
    for i in range(n_iters):
        if (i+1)%100==0:
            print('%d/%d:%.4f'%(i,n_iters,compare_L1(magnitude,MAG)))
        
        
        signal = stft_fn.inverse(magnitudes, angles).squeeze(1)
        MAG, f_angles_0 = stft_fn.transform(signal)
        g_angles_0 = f_angles_0-angles
        
        signal = stft_fn.inverse(magnitudes, angles_1).squeeze(1)
        MAG, f_angles_1 = stft_fn.transform(signal)
        g_angles_1 = f_angles_1-angles_1
        
        cof = (angles_1*100 - angles*100)/(g_angles_1*100-g_angles_0*100)
        cof[cof>1e4]=0
        cof[cof<-1e4]=0
        cof[torch.isnan(cof)]=0
#         print(cof)
        angles_new = angles_1 - g_angles_1 *cof
        angles=angles_1
        angles_1=angles_new
        
    return angles_new

phase_newton_iterative =  newton_iterative(magnitude, stft_fn, n_iters=500)
print(phase_newton_iterative.shape)
audio_newton_iterative = stft_fn.inverse(magnitude, phase_newton_iterative)
audio_newton_iterative = audio_newton_iterative.squeeze(0)
print(audio_newton_iterative.shape)

ipd.Audio(audio_newton_iterative[0].data.cpu().numpy(), rate=hparams.sampling_rate)

99/500:0.8125
199/500:0.7413
299/500:0.7189
399/500:0.7046
499/500:0.6988
torch.Size([1, 513, 386])
torch.Size([1, 98560])


In [9]:
def steffensen(magnitudes, stft_fn, n_iters=30, angles=None):
    
    if angles==None:
        angles = np.angle(np.exp(2j * np.pi * np.random.rand(*magnitudes.size())))
        angles = angles.astype(np.float32)
        angles = torch.autograd.Variable(torch.from_numpy(angles))

    signal = stft_fn.inverse(magnitudes, angles).squeeze(1)
    MAG, angles_1 = stft_fn.transform(signal)
        
    for i in range(n_iters):
        if (i+1)%100==0:
            print('%d/%d:%.4f'%(i,n_iters,compare_L1(magnitude,MAG)))
        
        signal = stft_fn.inverse(magnitudes, angles).squeeze(1)
        MAG, angles_1 = stft_fn.transform(signal)
        signal = stft_fn.inverse(magnitudes, angles_1).squeeze(1)
        MAG, angles_2 = stft_fn.transform(signal)
        cof = (angles_1*10 - angles*10)*(angles_1*10 - angles*10) / ((angles_2*100-2*angles_1*100+angles*100))
        cof[cof>1e4]=0
        cof[cof<-1e4]=0
        cof[torch.isnan(cof)]=0
#         print(cof)
        angles_new = angles - cof
        angles = angles_new
        
    return angles

# phase_newton_iterative =  steffensen(magnitude, stft_fn, n_iters=10, angles=phase_griffin_lim)
phase_newton_iterative =  steffensen(magnitude, stft_fn, n_iters=500)
print(phase_newton_iterative.shape)
audio_newton_iterative = stft_fn.inverse(magnitude, phase_newton_iterative)
audio_newton_iterative = audio_newton_iterative.squeeze(0)
print(audio_newton_iterative.shape)

ipd.Audio(audio_newton_iterative[0].data.cpu().numpy(), rate=hparams.sampling_rate)

99/500:0.1671
199/500:0.1361
299/500:0.1254
399/500:0.1193
499/500:0.1157
torch.Size([1, 513, 386])
torch.Size([1, 98560])


In [9]:
def adam_fix_point(magnitudes, stft_fn, n_iters=30, x=None):
    # Adam solve f(x)=x
    # Adam solve min: (f(x)-x)^2
    
    if x==None:
        x = np.angle(np.exp(2j * np.pi * np.random.rand(*magnitudes.size())))
        x = x.astype(np.float32)
        x = torch.tensor(torch.from_numpy(x), requires_grad=True)
    else:
        x = torch.tensor(x, requires_grad=True)

    optimizer = torch.optim.Adam([x], lr=1e-1)
    for i in range(n_iters):
        if (i+1)%100==0:
            print('%d/%d:%.4f loss=%.4f'%(i,n_iters,compare_L1(magnitude,MAG), loss.item()))
        
        
        signal = stft_fn.inverse(magnitudes, x).squeeze(1)
        MAG, fx = stft_fn.transform(signal)
        loss = torch.sum((fx-x)**2)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return x

phase_adam =  adam_fix_point(magnitude, stft_fn, n_iters=500, x= phase_griffin_lim_30)
print(phase_adam.shape)
audio_adam = stft_fn.inverse(magnitude, phase_adam)
audio_adam = audio_adam.squeeze(0)
print(audio_adam.shape)

ipd.Audio(audio_adam[0].data.cpu().numpy(), rate=hparams.sampling_rate)

/local/scratch/yji38/miniconda/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


99/500:0.7579 loss=62886.4787
199/500:0.7380 loss=51378.0177
299/500:0.7489 loss=49714.5035
399/500:0.7388 loss=38331.6529
499/500:0.7374 loss=44569.0197
torch.Size([1, 513, 386])
torch.Size([1, 98560])


In [10]:
def adam_fix_point(magnitudes, stft_fn, n_iters=30, x=None):
    # Adam solve f(x)=x
    # Adam solve min: (f(x)-x)^2
    
    if x==None:
        x = np.angle(np.exp(2j * np.pi * np.random.rand(*magnitudes.size())))
        x = x.astype(np.float32)
        x = torch.tensor(torch.from_numpy(x), requires_grad=True)
    else:
        x = torch.tensor(x, requires_grad=True)

    optimizer = torch.optim.Adam([x], lr=1e-1)
    for i in range(n_iters):
        if (i+1)%100==0:
            print('%d/%d:%.4f loss=%.4f'%(i,n_iters,compare_L1(magnitude,MAG), loss.item()))
        
        
        signal = stft_fn.inverse(magnitudes, x).squeeze(1)
        MAG, fx = stft_fn.transform(signal)
        loss = torch.sum((MAG - magnitudes)**2)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return x

phase_adam =  adam_fix_point(magnitude, stft_fn, n_iters=500, x= phase_griffin_lim_30)
print(phase_adam.shape)
audio_adam = stft_fn.inverse(magnitude, phase_adam)
audio_adam = audio_adam.squeeze(0)
print(audio_adam.shape)

ipd.Audio(audio_adam[0].data.cpu().numpy(), rate=hparams.sampling_rate)

/local/scratch/yji38/miniconda/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


99/500:0.5085 loss=7950.2913
199/500:0.4614 loss=7403.0576
299/500:0.4323 loss=6819.0071
399/500:0.4126 loss=6576.7696
499/500:0.4076 loss=6526.5718
torch.Size([1, 513, 386])
torch.Size([1, 98560])
